# Leitura dos dados

In [0]:
#Biblioteca para ler os dados de teste
import pandas as pd
#Leitura das instruções no formato TXT e convertendo para um DataFrame
teste = pd.read_csv("https://raw.githubusercontent.com/gilvandrocesardemedeiros/MIPS/master/teste.txt", sep = " ", 
            names = ["Instruction","Coluna1","Coluna2","Coluna3"])
#Retirando as vírgulas
for j in teste.columns:
  for i in teste.index:
    teste.loc[i, j] = teste.loc[i, j].replace(",", "")
#Instruções de teste
teste.head(7)

# Funções Auxiliares

In [2]:
#Função para converter binário em inteiro
def binToNum(binario):
  num = 0
  negativo = False
  for i in range(len(binario)):
    if binario[i] == "b":
      negativo = True
    else:
      num = num + int(binario[i]) * pow(2, len(binario) - i - 1)
  if negativo:
    num = -1 * num
  return num

#Teste para função para conversão 
binToNum("1111")

15

In [3]:
#Função que transforma o número binário em uma quantidade "numDigitos" de díígitos (acrescentando zeros à esquerda)
def binNormalizer(binario, numDigitos):
  while len(binario) < numDigitos:
    binario = "0" + binario
  return binario

#Teste para função
binNormalizer("101", 5)

'00101'

In [4]:
#Decodificando as instruções em códigos de 32 bits

#Dicionario que relaciona cada instrucao a um código de 6 bits (opcode)
dictInst = {"ADD" : "000001", "ADDI" : "000010", "SUB" : "000100"}

#Dicionário que relaciona cada registrador a um código binário de 5 bits
dictRegs = {"$zero" : "00000", "$t0" : "0" + bin(8)[2:], "$t1" :"0" +  bin(9)[2:], "$t2" :"0" +  bin(10)[2:], "$t3" :"0" +  bin(11)[2:], 
            "$t4" :"0" +  bin(12)[2:], "$t5" : "0" + bin(13)[2:], "$t6" : "0" + bin(14)[2:], "$t7" :"0" +  bin(15)[2:], "$s0" : bin(16)[2:], 
            "$s1" : bin(17)[2:], "$s2" : bin(18)[2:], "$s3" : bin(19)[2:],"$s4" : bin(20)[2:], "$s5" : bin(21)[2:], "$s6" : bin(22)[2:], 
            "$s7" : bin(23)[2:], "$t8" : bin(24)[2:], "$t9" : bin(25)[2:]}

#Funcao para decodificar códigos de teste em códigos binários
def decodificador(codigos):
  instrucoes = pd.DataFrame(index = codigos.index)
  instrucoes["Instrucoes"] = "NaN"
  for i in codigos.index:
    #Instrucoes tipo I
    if codigos.loc[i, "Instruction"] == "ADDI":
      #Identificando o valor imediato em binário a ser utilizado
      imediato = bin(int(codigos.loc[i, "Coluna3"]))[2:]
      #Garantindo que o valor imediato possua 16 bits
      imediato = binNormalizer(imediato, 16)
      #Identificando o registrador fonte e substituindo-o pelo código binário que o representa
      rs = dictRegs[codigos.loc[i, "Coluna2"]]
      #Identificando o registrador para o qual seráá atribuído o resultado da operação e substituindo-o pelo código binário que o representa
      rt = dictRegs[codigos.loc[i, "Coluna1"]]
      #Instrucoes tipo I: 6 bits (opcode) + 5 bits (rs) + 5 bits (rt) + 16 bits (imediato)
      instrucoes.loc[i, "Instrucoes"] = dictInst["ADDI"] + rs + rt + imediato

    #Instrucoes tipo R
    elif (codigos.loc[i, "Instruction"] == "ADD") or (codigos.loc[i, "Instruction"] == "SUB"):
      #Padrão de opcode para instrução R
      opcode = "000000"
      #Registrador para o qual será destinado o resultado da operação
      rd = dictRegs[codigos.loc[i, "Coluna1"]]
      #Primeiro registrador para fornecer valor para ser operado
      rs = dictRegs[codigos.loc[i, "Coluna2"]]
      #Segundo registrador para fornecer valor para ser operado
      rt = dictRegs[codigos.loc[i, "Coluna3"]]
      #Shamt
      shamt = "00000"
      #Funcao - "ADD" ou "SUB"
      funct = dictInst[codigos.loc[i, "Instruction"]]
      #Instrucoes tipo R: 6 bits (opcode) + 5 bits (rs) + 5 bits (rt) + 5 bits (rd) + 5 bits (shamt) + 6 bits (funct)
      instrucoes.loc[i, "Instrucoes"] = opcode + rs + rt + rd + shamt + funct
  return instrucoes

#Teste para o decodificador (código Assembly <-> Binário)
decodificador(teste)

,Instrucoes
0,00001000000100010000000000001010
1,00001000000100100000000000000010
2,00001000000100110000000000001000
3,00001000000101000000000000000110
4,00000010001100100100000000000001
5,00000010011101000100100000000001
6,00000001000010011000000000000100


# Blocos de DataPath

In [5]:
#Contador de programa
class PC:
  def __init__(self):
    self.cont = binNormalizer("0", 32)
    print("Contador de programa inicializado com valor " + self.cont)
  #Altera o valor do contador
  def setCont(self, value):
    self.cont = value
    print("Contador de programa alterado para valor: " + self.cont)

#Teste para contador de programa
contador = PC()
contador.setCont(binNormalizer(bin(10)[2:], 32))

Contador de programa inicializado com valor 00000000000000000000000000000000
Contador de programa alterado para valor: 00000000000000000000000000001010


In [6]:
#Bloco de instrução de memória 
class InstructionMemory:
  def __init__(self, teste):  
    self.AdressInstruction = binNormalizer("0", 32)
    print("Intruction Memory inicializado com instrução apontando para " + self.AdressInstruction)
    self.Instrucoes = decodificador(teste)
    print("Instruções lidas e codificadas como a seguir: ")
    print(self.Instrucoes)

  def loadInstructions(self, AdrInst):
    #Selecionando instrução apontada pelo PC
    instrucaoAtual = self.Instrucoes.loc[binToNum(AdrInst), "Instrucoes"]
    print("Instrução atual: " + instrucaoAtual)
    #Verificando se a instrução é do tipo R
    if instrucaoAtual[:6] == "000000":
      print("Instrução do tipo R")
      #Instrucoes tipo R: 6 bits (opcode) + 5 bits (rs) + 5 bits (rt) + 5 bits (rd) + 5 bits (shamt) + 6 bits (funct)
      self.rs = instrucaoAtual[6:11]
      print("rs: " + self.rs)
      self.rt = instrucaoAtual[11:16]
      print("rt: " + self.rt)
      self.rd = instrucaoAtual[16:21]
      print("rd: " + self.rd)
      self.schamt = instrucaoAtual[21:26]
      print("schamt: " + self.schamt)
      self.funct = instrucaoAtual[-6:]
      print("funct: " + self.funct)
    else:
      print("Instrução do tipo I")
      #Instrucoes tipo I: 6 bits (opcode) + 5 bits (rs) + 5 bits (rt) + 16 bits (imediato)
      self.rs = instrucaoAtual[6:11]
      print("rs: " + self.rs)
      self.rt = instrucaoAtual[11:16]
      print("rt: " + self.rt)
      self.imediato = instrucaoAtual[-16:]
      print("imediato: " + self.imediato)
      self.funct = instrucaoAtual[:6]
      print("funct: " + self.funct)


#Teste para instruction memory
instMemory = InstructionMemory(teste)
instMemory.loadInstructions(binNormalizer("5", 32))

Intruction Memory inicializado com instrução apontando para 00000000000000000000000000000000
Instruções lidas e codificadas como a seguir: 
                         Instrucoes
0  00001000000100010000000000001010
1  00001000000100100000000000000010
2  00001000000100110000000000001000
3  00001000000101000000000000000110
4  00000010001100100100000000000001
5  00000010011101000100100000000001
6  00000001000010011000000000000100
Instrução atual: 00000010011101000100100000000001
Instrução do tipo R
rs: 10011
rt: 10100
rd: 01001
schamt: 00000
funct: 000001


In [7]:
class Registers:
  def __init__(self):
    #Registradores declarados conforme seus códigos binários de endereço
    self.registradores = {
        "00000" : binNormalizer("0", 32), #$zero
        "0" + bin(8)[2:] : binNormalizer("0", 32), #$t0
        "0" +  bin(9)[2:] : binNormalizer("0", 32), #$t1
        "0" +  bin(10)[2:] : binNormalizer("0", 32), #$t2
        "0" +  bin(11)[2:] : binNormalizer("0", 32), #$t3
        "0" +  bin(12)[2:] : binNormalizer("0", 32), #$t4
        "0" + bin(13)[2:] : binNormalizer("0", 32), #$t5
        "0" + bin(14)[2:]: binNormalizer("0", 32), #$t6
        "0" +  bin(15)[2:] : binNormalizer("0", 32), #$t7
        bin(16)[2:] : binNormalizer("0", 32), #$s0 
        bin(17)[2:] : binNormalizer("0", 32), #$s1
        bin(18)[2:] : binNormalizer("0", 32), #$s2
        bin(19)[2:] : binNormalizer("0", 32), #$s3
        bin(20)[2:] : binNormalizer("0", 32), #$s4
        bin(21)[2:] : binNormalizer("0", 32), #$s5
        bin(22)[2:] : binNormalizer("0", 32), #$s6 
        bin(23)[2:] : binNormalizer("0", 32), #$s7
        bin(24)[2:] : binNormalizer("0", 32), #$t8
        bin(25)[2:] : binNormalizer("0", 32) #$t9
    }
    
#Teste para Registers
reg = Registers()
print("Valor armazenado em 01000 ($t0): " + reg.registradores["01000"])

Valor armazenado em 01000 ($t0): 00000000000000000000000000000000


In [8]:
class Alu:
  def __init__(self):
    self.input1 = binNormalizer("0", 32)
    self.input2 = binNormalizer("0", 32)
    self.funct = binNormalizer("0", 6)
    self.output = binNormalizer("0", 32)
    print("Alu inicializada com entrada 1 = " + self.input1 + " e entrada 2 = " + self.input2)
    print("Código de operação inicial: " + self.funct)
    print("Resultado inicial: " + self.output)
  
  def setParameters(self, in1, in2):
    self.input1 = in1
    self.input2 = in2
    print("Alu lendo dados " + in1 + " e " + in2)

  def operation(self, op):
    #Operacao de ADD
    if op == "000001":
      print("Operação: ADD, com entradas (convertidas para decimal): " + str(binToNum(self.input1)) + " e " + str(binToNum(self.input2)))
      resultadoNumerico = binToNum(self.input1) + binToNum(self.input2)
      self.output = binNormalizer(bin(resultadoNumerico)[2:], 32)
      print("Resultado da Alu: Decimal (" + str(resultadoNumerico) + "), Binário: (" + self.output + ")")
    
    #Operacao de ADDI
    if op == "000010":
      print("Operação: ADDI, com entradas (convertidas para decimal): " + str(binToNum(self.input1)) + " e " + str(binToNum(self.input2)))
      resultadoNumerico = binToNum(self.input1) + binToNum(self.input2)
      self.output = binNormalizer(bin(resultadoNumerico)[2:], 32)
      print("Resultado da Alu: Decimal (" + str(resultadoNumerico) + "), Binário: (" + self.output + ")")
    
    #Operacao de SUB
    if op == "000100":
      print("Operação: SUB, com entradas (convertidas para decimal): " + str(binToNum(self.input1)) + " e " + str(binToNum(self.input2)))
      resultadoNumerico = binToNum(self.input1) - binToNum(self.input2)
      self.output = binNormalizer(bin(resultadoNumerico)[2:], 32)
      print("Resultado da Alu: Decimal (" + str(resultadoNumerico) + "), Binário: (" + self.output + ")")

#Teste para Alu
reg = Alu()
reg.setParameters(binNormalizer("01", 32), binNormalizer("10", 32))
reg.operation("000001")

Alu inicializada com entrada 1 = 00000000000000000000000000000000 e entrada 2 = 00000000000000000000000000000000
Código de operação inicial: 000000
Resultado inicial: 00000000000000000000000000000000
Alu lendo dados 00000000000000000000000000000001 e 00000000000000000000000000000010
Operação: ADD, com entradas (convertidas para decimal): 1 e 2
Resultado da Alu: Decimal (3), Binário: (00000000000000000000000000000011)


In [9]:
#Objeto "Add"
class Add:
  def __init__(self):
    self.contador = binNormalizer("0", 32)
    self.input1 = binNormalizer("1", 32) #Valor de incremento. Sempre incrementar de 1 em 1
    self.output = binNormalizer("0", 32)
    print("Bloco Add adicionado, com entrada do contador = " + self.contador + ", entrada 1 = " + self.input1 + " e saída: " + self.output)

  def operation(self, cont):
    print("ADD entrando em operação")
    self.contador = cont
    print("Estado atual do contador: " + self.contador)
    resultadoNumerico = binToNum(self.contador) + binToNum(self.input1)
    self.output = binNormalizer(bin(resultadoNumerico)[2:], 32)
    print("Saída de ADD: " + self.output)

#Teste para Add
add = Add()
add.operation(binNormalizer("01", 32))

Bloco Add adicionado, com entrada do contador = 00000000000000000000000000000000, entrada 1 = 00000000000000000000000000000001 e saída: 00000000000000000000000000000000
ADD entrando em operação
Estado atual do contador: 00000000000000000000000000000001
Saída de ADD: 00000000000000000000000000000010


# DataPath

In [0]:
class DataPath:
  def __init__(self):
    #Inicializando blocos
    self.pc = PC()
    self.reg = Registers()
    self.alu = Alu()
    self.add = Add()
  
  def operateInstructions(self, teste):
    #Passando as instruções para a memória
    self.instmemory = InstructionMemory(teste)
    #Percorrendo todas as instruções
    for i in teste.index:
      self.instmemory.loadInstructions(self.pc.cont)
      #Instruções tipo R
      if self.instmemory.funct == "000001" or self.instmemory.funct == "000100":
        #Mandando para a ALU os dados nos registradores rs e rt
        self.alu.setParameters(self.reg.registradores[self.instmemory.rs], self.reg.registradores[self.instmemory.rt])
        #Mandando para a ALU a operação a ser realizada
        self.alu.operation(self.instmemory.funct)
        #Guardando no registrador rd o resultado da operação da ALU
        print("DEBUG IS ON THE TABLE " +self.instmemory.rd)
        self.reg.registradores[self.instmemory.rd] = self.alu.output
        #Valor armazenado no registrador rd
        print("Armazenando " + self.reg.registradores[self.instmemory.rd] + " no registrador " + self.instmemory.rd)
        #Incrementando "1" no contador e mandando para a saída do Add
        self.add.operation(self.pc.cont)
        #Passando a saída do Add para entrada do PC
        self.pc.setCont(self.add.output)
      else: #Instruções tipo I
        #Mandando para a ALU os dados no registrador rs e o imediato
        self.alu.setParameters(self.reg.registradores[self.instmemory.rs], self.instmemory.imediato)
        #Mandando para a ALU a operação a ser realizada
        self.alu.operation(self.instmemory.funct)
        #Guardando no registrador rt o resultado da operação da ALU
        self.reg.registradores[self.instmemory.rt] = self.alu.output
        #Valor armazenado no registrador rt
        print("Armazenando " + self.reg.registradores[self.instmemory.rt] + " no registrador " + self.instmemory.rt)
        #Incrementando "1" no contador e mandando para a saída do Add
        self.add.operation(self.pc.cont)
        #Passando a saída do Add para entrada do PC
        self.pc.setCont(self.add.output)

  def visualizarRegistradores(self):
    invDictRegs = {"00000" : "$zero",
                     "0" + bin(8)[2:] :  "$t0",
                     "0" +  bin(9)[2:] :  "$t1", 
                     "0" +  bin(10)[2:] : "$t2", 
                     "0" +  bin(11)[2:] : "$t3", 
                     "0" +  bin(12)[2:] : "$t4",
                     "0" + bin(13)[2:] : "$t5",
                     "0" + bin(14)[2:] : "$t6",
                     "0" +  bin(15)[2:] : "$t7", 
                     bin(16)[2:] : "$s0", 
                     bin(17)[2:] : "$s1", 
                     bin(18)[2:] : "$s2", 
                     bin(19)[2:] : "$s3",
                     bin(20)[2:] : "$s4", 
                     bin(21)[2:] : "$s5", 
                     bin(22)[2:] : "$s6", 
                     bin(23)[2:] : "$s7", 
                     bin(24)[2:] : "$t8", 
                     bin(25)[2:] : "$t9"}
    for i in self.reg.registradores.keys():
      print(invDictRegs[i] + " = " + str(binToNum(self.reg.registradores[i])))

In [11]:
dp = DataPath()

Contador de programa inicializado com valor 00000000000000000000000000000000
Alu inicializada com entrada 1 = 00000000000000000000000000000000 e entrada 2 = 00000000000000000000000000000000
Código de operação inicial: 000000
Resultado inicial: 00000000000000000000000000000000
Bloco Add adicionado, com entrada do contador = 00000000000000000000000000000000, entrada 1 = 00000000000000000000000000000001 e saída: 00000000000000000000000000000000


In [12]:
dp.operateInstructions(teste)

Intruction Memory inicializado com instrução apontando para 00000000000000000000000000000000
Instruções lidas e codificadas como a seguir: 
                         Instrucoes
0  00001000000100010000000000001010
1  00001000000100100000000000000010
2  00001000000100110000000000001000
3  00001000000101000000000000000110
4  00000010001100100100000000000001
5  00000010011101000100100000000001
6  00000001000010011000000000000100
Instrução atual: 00001000000100010000000000001010
Instrução do tipo I
rs: 00000
rt: 10001
imediato: 0000000000001010
funct: 000010
Alu lendo dados 00000000000000000000000000000000 e 0000000000001010
Operação: ADDI, com entradas (convertidas para decimal): 0 e 10
Resultado da Alu: Decimal (10), Binário: (00000000000000000000000000001010)
Armazenando 00000000000000000000000000001010 no registrador 10001
ADD entrando em operação
Estado atual do contador: 00000000000000000000000000000000
Saída de ADD: 00000000000000000000000000000001
Contador de programa alterado para v

In [13]:
dp.visualizarRegistradores()

$zero = 0
$t0 = 12
$t1 = 14
$t2 = 0
$t3 = 0
$t4 = 0
$t5 = 0
$t6 = 0
$t7 = 0
$s0 = -2
$s1 = 10
$s2 = 2
$s3 = 8
$s4 = 6
$s5 = 0
$s6 = 0
$s7 = 0
$t8 = 0
$t9 = 0
